In [1]:
%load_ext autoreload
%autoreload 2

import json

from grants_tagger.slim.mesh_xlinear import train_and_evaluate

In [2]:
parameters = {
    'ngram_range': (1, 1), 
    'beam_size': 30, 
    'only_topk': 200, 
    'min_weight_value': 0.1,
    'max_features': 400_000
}

In [ ]:
!curl -X POST -H 'Content-type: application/json' --data "{'text': 'Hi <$SLACK_USER>, training has started'}" $SLACK_HOOK

model = train_and_evaluate(
    train_data_path='../data/processed/train_mesh2021.jsonl',
    test_data_path='../data/processed/test_mesh2021.jsonl',
    label_binarizer_path='../models/xlinear-full-test/label_binarizer.pkl',
    model_path='../models/xlinear-toy',
    results_path='../results/xlinear-full-test',
    full_report_path='../results/mesh_xlinear_full_full_report.json'
)

!curl -X POST -H 'Content-type: application/json' --data "{'text': 'Hi <$SLACK_USER>, training has finished'}" $SLACK_HOOK

ok

In [ ]:
with open('../results/mesh_xlinear.json', 'r') as f:
    results = json.load(f)

In [ ]:
results